# Estimate Binomial draw probabilility

In [1]:
using DynamicHMCModels

Define a structure to hold the data.

In [2]:
struct BernoulliProblem
    "Total number of draws in the data."
    n::Int
    "Number of draws `==1` in the data"
    s::Vector{Int}
end;

Make the type callable with the parameters *as a single argument*.

In [3]:
function (problem::BernoulliProblem)((α, )::NamedTuple{(:α, )})
    @unpack n, s = problem        # extract the data
    loglikelihood(Binomial(n, α), s)
end

Create the data and complete setting up the problem.

In [4]:
obs = rand(Binomial(9, 2/3), 1)
p = BernoulliProblem(9, obs)
p((α = 0.5, ))

-2.6548056865833978

Write a function to return properly dimensioned transformation.

In [5]:
problem_transformation(p::BernoulliProblem) =
    as((α = as𝕀, ),  )

problem_transformation (generic function with 1 method)

Use a flat priors (the default, omitted) for α

In [6]:
P = TransformedLogDensity(problem_transformation(p), p)
∇P = ADgradient(:ForwardDiff, P);

#import Zygote
#∇P = ADgradient(:Zygote, P);

Sample

In [7]:
chain, NUTS_tuned = NUTS_init_tune_mcmc(∇P, 1000)

MCMC, adapting ϵ (75 steps)
3.2e-5 s/step ...done
MCMC, adapting ϵ (25 steps)
3.2e-5 s/step ...done
MCMC, adapting ϵ (50 steps)
3.5e-5 s/step ...done
MCMC, adapting ϵ (100 steps)
3.2e-5 s/step ...done
MCMC, adapting ϵ (200 steps)
2.6e-5 s/step ...done
MCMC, adapting ϵ (400 steps)
2.7e-5 s/step ...done
MCMC, adapting ϵ (50 steps)
3.5e-5 s/step ...done
MCMC (1000 steps)
5.5e-5 s/step ...done


(NUTS_Transition{Array{Float64,1},Float64}[NUTS_Transition{Array{Float64,1},Float64}([1.0328655247033827], -3.2374385234471186, 1, DynamicHMC.AdjacentTurn, 0.9875355398706155, 3), NUTS_Transition{Array{Float64,1},Float64}([0.887552808224404], -2.872930953640701, 2, DynamicHMC.DoubledTurn, 0.9990108145348179, 3), NUTS_Transition{Array{Float64,1},Float64}([1.4908224999609556], -3.240441707529592, 2, DynamicHMC.DoubledTurn, 0.9160725538494452, 3), NUTS_Transition{Array{Float64,1},Float64}([1.4797851669566797], -3.6887746008342885, 2, DynamicHMC.DoubledTurn, 0.9452034819163977, 3), NUTS_Transition{Array{Float64,1},Float64}([0.923771958044846], -3.155302281015916, 2, DynamicHMC.DoubledTurn, 0.9758107224010629, 3), NUTS_Transition{Array{Float64,1},Float64}([1.3655683012809976], -3.094929852695037, 2, DynamicHMC.DoubledTurn, 0.9472331084426999, 3), NUTS_Transition{Array{Float64,1},Float64}([1.353715149291812], -3.534793313547187, 2, DynamicHMC.DoubledTurn, 0.9441643681033716, 3), NUTS_Transit

To get the posterior for ``α`` use `get_position` and then transform back.

In [8]:
posterior = TransformVariables.transform.(Ref(problem_transformation(p)), get_position.(chain));

Extract the parameter.

In [9]:
posterior_α = first.(posterior);

check the effective sample size

In [10]:
ess_α = effective_sample_size(posterior_α)

334.688636159288

NUTS-specific statistics

In [11]:
NUTS_statistics(chain)

Hamiltonian Monte Carlo sample of length 1000
  acceptance rate mean: 0.89, min/25%/median/75%/max: 0.03 0.86 0.97 1.0 1.0
  termination: AdjacentTurn => 24% DoubledTurn => 76%
  depth: 1 => 61% 2 => 39%


check the mean

In [12]:
mean(posterior_α)

0.7333630947128722

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*